In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x37ed080
Welcome to JupyROOT 6.26/10


In [2]:
ROOT.EnableImplicitMT(6)

In [3]:
%jsroot on

In [4]:
ROOT.gSystem.Load("libedm4hep")
ROOT.gSystem.Load("libpodio")
ROOT.gSystem.Load("libFCCAnalyses")

0

In [5]:
_edm  = ROOT.edm4hep.ReconstructedParticleData()
_pod  = ROOT.podio.ObjectID()
_fcc  = ROOT.dummyLoader

In [6]:
files = (
    "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pL.I500105.0_edm4hep.root",
    "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root",
    "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pR.I500107.0_edm4hep.root",
    "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pL.I500108.0_edm4hep.root"
)
#df = ROOT.RDataFrame("events", "test_4f_sw_sl_eL_pR_edm4hep.root")
#df = ROOT.RDataFrame("events", "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root")
#df = ROOT.RDataFrame("events", "E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pL.I500108.0_edm4hep.root")
df = ROOT.RDataFrame("events", files)

In [7]:
df.Describe()

Dataframe from TChain events in files
  E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pL.I500105.0_edm4hep.root
  E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eL.pR.I500106.0_edm4hep.root
  E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pR.I500107.0_edm4hep.root
  E250-SetA.P4f_sw_sl.Gwhizard-2_8_5.eR.pL.I500108.0_edm4hep.root

Property                Value
--------                -----
Columns in total           39
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                  Type                                            Origin
------                                  ----                                            ------
EventHeader                             ROOT::VecOps::RVec<edm4hep::EventHeaderData>    Dataset
EventHeader.eventNumber                 ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.runNumber                   ROOT::VecOps::RVec<Int_t>                       Dataset
EventHeader.timeStamp                 

In [8]:
%%cpp
ROOT::VecOps::RVec<edm4hep::MCParticleData> get_enu(ROOT::VecOps::RVec<edm4hep::MCParticleData> in) {
    ROOT::VecOps::RVec<edm4hep::MCParticleData> res;
    for (size_t i = 0; i <= in.size(); i++) {
        const auto& mc = in[i];
        const auto& mc_next = in[i+1];
        if (abs(mc.PDG) == 11 && abs(mc_next.PDG) == 12) {
            res.push_back(mc);
            res.push_back(mc_next);
            break;
        }
    }
    return res;
}


In [9]:
%%cpp
ROOT::VecOps::RVec<edm4hep::MCParticleData> get_qq(ROOT::VecOps::RVec<edm4hep::MCParticleData> in) {
    ROOT::VecOps::RVec<edm4hep::MCParticleData> res;
    for (size_t i = 0; i <= in.size(); i++) {
        const auto& mc = in[i];
        const auto& mc_next = in[i+1];
        if (abs(mc.PDG) <= 5 && abs(mc_next.PDG) <= 5) {
            res.push_back(mc);
            res.push_back(mc_next);
            break;
        }
    }
    return res;
}

In [10]:
def defines(df):
       df2 = (df
              .Define("enu", "get_enu(MCParticles)")
              .Define("enu_p", "FCCAnalyses::MCParticle::get_p(enu)")
              .Define("enu_theta", "FCCAnalyses::MCParticle::get_theta(enu)")
              .Define("e", "enu[0]")
              .Define("nu", "enu[1]")
              .Define("e_p", "enu_p[0]")
              .Define("nu_p", "enu_p[1]")
              .Define("e_theta", "enu_theta[0]")
              .Define("nu_theta", "enu_theta[1]")
              .Define("e_seen", "abs(cos(e_theta)) < 0.989") # no silicon hits in ild below 8.5 deg
              .Define("enu_tlv", "FCCAnalyses::MCParticle::get_tlv(enu)")
              .Define("enu_W_tlv", "enu_tlv[0] + enu_tlv[1]")
              .Define("enu_invM", "(enu_tlv[0] + enu_tlv[1]).M()")
              .Define("qq", "get_qq(MCParticles)")
              .Define("qq_tlv", "FCCAnalyses::MCParticle::get_tlv(qq)")
              .Define("qq_W_tlv", "qq_tlv[0] + qq_tlv[1]")
              .Define("qq_invM", "qq_W_tlv.M()")
              .Define("qq_W_theta", "qq_W_tlv.Theta()")
              .Define("cos_theta_W", "cos(qq_W_theta)")
              .Define("e_tlv", "enu_tlv[0]")
              .Define("e_lv", "ROOT::Math::PxPyPzEVector(e_tlv.Px(), e_tlv.Py(), e_tlv.Pz(), e_tlv.E())")
              .Define("enu_lv", "ROOT::Math::PxPyPzEVector(enu_W_tlv.Px(), enu_W_tlv.Py(), enu_W_tlv.Pz(), enu_W_tlv.E())")
              .Define("qq_W_lv", "ROOT::Math::PxPyPzEVector(qq_W_tlv.Px(), qq_W_tlv.Py(), qq_W_tlv.Pz(), qq_W_tlv.E())")
              .Define("WW_lv", "qq_W_lv + enu_lv")
              .Define("e_enu_cms_lv", "ROOT::Math::Boost(enu_lv.BoostToCM())(e_lv)") # boost e to cms of enu
              .Define("decay_theta", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), enu_lv.Vect())")
              #.Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, e.charge).Cross(enu_lv.Vect().Unit()))")
              #.Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, e.charge))")
              .Define("decay_phi", "ROOT::Math::VectorUtil::Angle(e_enu_cms_lv.Vect(), ROOT::Math::XYZVector(0, 0, 1))")
              .Define("decay_cos_theta", "cos(decay_theta)")
              )
       return df2

In [11]:
df2 = defines(df)

In [12]:
#df2 = df2.Filter("abs(cos(e_theta)) < 0.98")

In [22]:
h = {}
h["e_p"] = df2.Histo1D("e_p")
h["nu_p"] = df2.Histo1D("nu_p")
h["e_theta"] = df2.Histo1D("e_theta")
h["nu_theta"] = df2.Histo1D("nu_theta")
h["enu_invM"] = df2.Histo1D("enu_invM")
h["qq_invM"] = df2.Histo1D("qq_invM")
h["cos_theta_Wp"] = df2.Filter("e.charge == -1").Histo1D(("", ";e^{-} cos #theta_{W}", 50, -1, 1), "cos_theta_W")
h["cos_theta_Wm"] = df2.Filter("e.charge == +1").Histo1D(("", ";e^{+} cos #theta_{W}", 50, -1, 1), "cos_theta_W")
h["cos_theta_W"] = df2.Histo1D(("", ";cos #theta_{W}", 50, -1, 1), "cos_theta_W")
h["decay_cos_theta_em"] = df2.Filter("e.charge == -1").Histo1D(("", ";e^{-} cos #theta_{l}", 50, -1, 1), "decay_cos_theta")
h["decay_cos_theta_ep"] = df2.Filter("e.charge == +1").Histo1D(("", ";e^{+} cos #theta_{l}", 50, -1, 1), "decay_cos_theta")
h["decay_cos_theta"] = df2.Histo1D(("", ";cos #theta_{l}", 50, -1, 1), "decay_cos_theta")
h["decay_phi_em"] = df2.Filter("e.charge == -1").Histo1D(("", ";e^{-} #phi_{l} [rad]", 50, 0, 3.142), "decay_phi")
h["decay_phi_ep"] = df2.Filter("e.charge == +1").Histo1D(("", ";e^{+} #phi_{l} [rad]", 50, 0, 3.142), "decay_phi")
h["decay_phi"] = df2.Histo1D(("", ";#phi_{l} [rad]", 50, 0, 3.142), "decay_phi")

In [23]:
#h["enu_invM_log"] = h["enu_invM"].Clone()

In [24]:
c = {}
for v in h:
    c[v] = ROOT.TCanvas()
    h[v].Draw()
    if (v.endswith("_log")):
        c[v].SetLogy()
    c[v].Draw()
    #c[v].SaveAs(f"plots/{v}.png")
    

